# Test dataset

## Setting up environment

In [1]:
import os
from glob import glob

import dask
import dask.bag as db
import dask.dataframe as dd
from distributed import Client
from dask_jobqueue import SLURMCluster

from IPython.display import display
import matplotlib.pyplot as plt

import logging
import pickle

from lenskit.batch import MultiEval, predict, recommend
from lenskit.crossfold import partition_users, SampleN
from lenskit.algorithms import basic, als, svd, bias
from lenskit import topn, util, Recommender, batch
from lenskit.topn import precision, ndcg, recall

import pandas as pd
import numpy as np
from scipy import stats
import binpickle

import argparse
import time

from tqdm.auto import tqdm

In [2]:
# Set LOCAL to True for single-machine execution while developing
# Set LOCAL to False for cluster execution
LOCAL = True

if LOCAL:
    # This line creates a single-machine dask client
    client = Client()
else:    
    # This line creates a SLURM cluster dask and dask client
    # Logging outputs will be stored in /scratch/{your-netid}
    
    cluster = SLURMCluster(memory='4GB', cores=2, python='/scratch/work/public/dask/bin/python', 
                               local_directory='/tmp/{}/'.format(os.environ['SLURM_JOB_USER']),
                               job_extra=['--output=/scratch/{}/slurm-%j.out'.format(os.environ['SLURM_JOB_USER'])])

    cluster.submit_command = 'slurm'
    cluster.scale(100)

    display(cluster)
    client = Client(cluster)

display(client)

Client Scheduler: tcp://127.0.0.1:39605 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 1 Cores: 1 Memory: 17.18 GB


## Load data

In [3]:
# use the 100 subtrain version for tuning
train_dir = '/scratch/zh2095/quarantini/cf_train.parquet'
test_dir = '/scratch/zh2095/quarantini/cf_test.parquet'

In [4]:
start = time.time()

df_train = pd.read_parquet(train_dir)
df_test = pd.read_parquet(test_dir)

print('took %f s'%(time.time()-start))

took 22.115638 s


In [5]:
df_train.rename(columns = {'user_id':'user', 'track_id':'item', 'count':'rating'}, inplace = True)
df_test.rename(columns = {'user_id':'user', 'track_id':'item', 'count':'rating'}, inplace = True)

In [6]:
df_train.reset_index(drop=True, inplace=True)

## Run the tuned model on the test dataset

In [8]:
# best configuration
best_features = 100
best_reg = 1
best_weight = 10
iterations = 20
use_ratings = True

data = (df_train, df_test)

In [9]:
start = time.time()

eval = MultiEval('result_test', recommend=500, save_models=False)  #set save_models=True when running on the full dataset
eval.add_datasets(data, name='Song')   
best_mdl = [als.ImplicitMF(features=best_features, iterations=iterations, reg=best_reg, 
                           use_ratings = use_ratings, weight=best_weight, method='cg', progress=tqdm)]
eval.add_algorithms(best_mdl, attrs=['features'], name='ImplicitMF')
eval.run(progress = tqdm)

print('took %f s'%(time.time()-start))

initializing deprecated MultiEval


  0%|          | 0/1 [00:00<?, ?it/s]

ImplicitMF:   0%|          | 0/20 [00:00<?, ?it/s]

took 6607.842613 s


# Final Result

In [8]:
recs_test = pd.read_parquet('result_test/recommendations.parquet')

In [11]:
rla = topn.RecListAnalysis()
# rla.add_metric(topn.ndcg)
# rla.add_metric(topn.recall)
rla.add_metric(precision, k=500)
rla.add_metric(ndcg, k=500)
rla.add_metric(recall, k=500)

raw_metrics_test = rla.compute(recs_test, df_test, include_missing=False)

In [12]:
metrics_test = raw_metrics_test.drop(columns=['nrecs']).fillna(0).mean()
metrics_test

precision    0.012088
ndcg         0.209590
recall       0.450778
dtype: float64

## Calculate MAP manually

In [25]:
start = time.time()

AP = []
for k in range(1,501):
    rla_map = topn.RecListAnalysis()
    rla_map.add_metric(precision, k=k)
    raw_metrics_map = rla_map.compute(recs_test, df_test, include_missing=False)
    metrics_map = raw_metrics_map.drop(columns=['nrecs']).fillna(0).mean()
    AP.append(metrics_map[0])

MAP = np.mean(AP)
print(f'MAP@{k} = {MAP}')
print('took %f s'%(time.time()-start))

MAP@500 = 0.028605261753119414
took 2662.503888 s
